In [1]:
import pandas as pd
import string
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
import numpy as np

In [25]:

# load the data from the CSV file

#load trianing data
data = pd.read_csv('storfil.csv',sep=',',encoding = 'latin-1')

#load disco code names
format1 = pd.read_csv('csv_da.csv',sep=';',encoding = 'utf-8')[['KODE','TITEL']]
format1['B2StilD'] = format1['KODE']


#DROP NA
data.dropna(subset=['B2StilD'], inplace =True)


#remove dots from DISCO code
data.B2StilD = data.B2StilD.astype(str)
data.B2StilD = data.B2StilD.str.replace('.', '')

#Comine disco code names with training data
format1['KODE'] = format1['KODE'].astype(str)
format1['KODE'] = format1['KODE'].str.slice(stop=4)
format_grouped = format1.groupby('KODE').agg({'TITEL':''.join}).reset_index()
data['B2StilD'] = data['B2StilD'].map(format_grouped.set_index('KODE')['TITEL'])


#make B2stil + B2stilA
data['B2Stilplus'] = data['B2Stil'] +' '+ data['B2StilA']


#format_dict = format_grouped.set_index('KODE').to_dict()
#data['B2StilD'] = data['B2StilD'].apply(lambda x: format_dict[x])

#data = data.merge(format,on='B2StilD',how='left')
#print(data['B2Stil'])

C:\Users\villa\AppData\Local\Temp\ipykernel_23564\3363721854.py:17: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  data.B2StilD = data.B2StilD.str.replace('.', '')


In [26]:

# load the data from the CSV file
data.dropna(subset=['B2StilD'], inplace =True)
# define a list of stop words
stop_words = set(stopwords.words('danish'))

# define a stemmer
stemmer = PorterStemmer()


In [27]:

# define a function to preprocess the text data
def preprocess_text(text):
    # convert text to lowercase
    text = text.lower()
    # remove punctuation
    text = text.translate(str.maketrans('', '', string.punctuation))
    # remove numbers
    text = ''.join([i for i in text if not i.isdigit()])
    # tokenize the text into words
    words = text.split()
    # remove stop words
    #words = [word for word in words if word not in stop_words]
    # perform stemming
    words = [stemmer.stem(word) for word in words]
    # join the words back into a single string
    text = ' '.join(words)
    return text

# preprocess the text data
#data['text'] = data['text'].apply(preprocess_text)


In [34]:
datab2stil =pd.DataFrame(data['B2Stilplus'])

new_sentence = {'B2Stilplus': 'danser jeg danser'}
datab2stil = pd.DataFrame(datab2stil.append(new_sentence, ignore_index=True))


print(datab2stil)
#print(datappend.iloc[:-1])


                                               B2Stilplus
0       MASKINOPERATØR står ved en maskine i produktio...
1         DISPONENT Fordeler arbejdsopgaver på chauffører
2       Lastbilchauffør - national transport Godstrans...
3       sagsbehandler - socialrådgiver Vejledning af b...
4              BIOANA Chef for nuklear medicinsk afdeling
...                                                   ...
148983                         adjunkt research, teaching
148984  maskinfører Kører gravmaskine og diverse andre...
148985  social og sundhedshjælper Passer og plejer æld...
148986             sælger sælger og laver mad i pølsevogn
148987                                  danser jeg danser

[148988 rows x 1 columns]


C:\Users\villa\AppData\Local\Temp\ipykernel_23564\1314178322.py:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  datab2stil = pd.DataFrame(datab2stil.append(new_sentence, ignore_index=True))


In [35]:

# preprocess the text data
datab2stil['text1'] = datab2stil['B2Stilplus'].astype(str).apply(preprocess_text)
#Not in use atm
#data['text1'] = data['B2Stil'].astype(str).apply(preprocess_text)

#data['text2'] = data['B2StilA'].astype(str).apply(preprocess_text)

#data['text3'] = data['B2StilD'].astype(str).apply(preprocess_text)

#data['combined'] = data.apply(lambda row: row['text1']+ '' + row['text2'],axis=1)

#data['random'] = np.random.randint(0,100)

print(datab2stil['text1'].iloc[-1])

danser jeg danser


In [36]:

# convert the text data into numerical features using TF-IDF
tfidf_vectorizer = TfidfVectorizer()

#tfidf_features1 = tfidf_vectorizer.fit_transform(data['text1'])
#tfidf_features2 = tfidf_vectorizer.fit_transform(data['text2'])
#tfidf_featuresY = tfidf_vectorizer.fit_transform(data['text3'])
tfidf_features4 = tfidf_vectorizer.fit_transform(datab2stil['text1'])

# Get the indices of the last row
last_doc_idx = tfidf_features4.shape[0] - 1

#capture last row
tfidf_newdata = tfidf_features4[-1, :]

# Remove the last row
tfidf_features4 = tfidf_features4[:last_doc_idx, :]


print(tfidf_newdata)



  (0, 6736)	0.9724369571259232
  (0, 17072)	0.23316595895558045


In [17]:
#Fjern nye sætninger:



# split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(tfidf_features4, data['B2StilD'], shuffle=False, stratify=None, test_size=0.2)


In [32]:

# train an SVM model on the training data
svm_model = SVC(kernel='linear')
svm_model.fit(X_train, y_train)

# test the SVM model on the testing data
accuracy = svm_model.score(X_test, y_test)
print("Accuracy:", accuracy)

Accuracy: 0.6486341365192295


In [37]:
svm_model.predict(tfidf_newdata)

ValueError: X has 43667 features, but SVC is expecting 78626 features as input.